![alt text](https://media.licdn.com/dms/image/C4E0BAQHXlPFZhl6kxQ/company-logo_200_200/0?e=2159024400&v=beta&t=S3U_G37BA8ldc_4ur5d0C-j7NHdEeCq-YLOqabikA5E)

# Importing Kaggle Dataset and Pre-Processing!




## configuring kaggle and downloading kaggle dataset

In [0]:
from google.colab import files
!mkdir ../root/.kaggle/
!cp ../content/kaggle.json ../root/.kaggle/
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6
# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 /root/.kaggle/kaggle.json
#!kaggle datasets download -d paramaggarwal/fashion-product-images-dataset
!kaggle datasets download -d paramaggarwal/fashion-product-images-small

mkdir: cannot create directory ‘../root/.kaggle/’: File exists
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)
Processing /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674/kaggle-1.5.6-cp36-none-any.whl
fashion-product-images-small.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
import os 
print(os.listdir("../content/"))

['.config', 'images', 'sample.jpg', 'fashion-product-images-small.zip', 'sample2.jpg', 'styles.csv', 'img2.jpg', 'img1.jpg', 'weights', 'topwear.jpg', 'myresultingzips.zip', 'kaggle.json', 'myntradataset', 'sample_data']


In [0]:
!unzip fashion-product-images-small.zip

Archive:  fashion-product-images-small.zip
replace images/10000.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Preparing the Dataset for Training and Testing!

## data prep

In [4]:
import pandas as pd
import os 
DATASET_PATH = "/content/myntradataset"
print(os.listdir(DATASET_PATH))

['images', 'styles.csv']


In [5]:
df = pd.read_csv(DATASET_PATH + "/" + "styles.csv", nrows=5000, error_bad_lines=False)
df['image'] = df.apply(lambda row: str(row['id']) + ".jpg", axis=1)
df = df.sample(frac=1).reset_index(drop=True)
df.head(10)

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,13472,Women,Apparel,Dress,Dresses,White,Summer,2011,Casual,United Colors of Benetton Women Solid White Dress,13472.jpg
1,47539,Women,Footwear,Shoes,Flats,White,Winter,2012,Casual,Portia Women White Flats,47539.jpg
2,26155,Men,Apparel,Bottomwear,Trousers,Grey,Summer,2012,Formal,John Miller Men Striped Grey Trousers,26155.jpg
3,53137,Women,Footwear,Shoes,Flats,Brown,Winter,2015,Casual,Catwalk Women Brown Sandals,53137.jpg
4,57932,Men,Apparel,Topwear,Tshirts,White,Summer,2012,Casual,French Connection Men White T-shirt,57932.jpg
5,33027,Women,Apparel,Topwear,Kurtas,Blue,Summer,2012,Ethnic,Mother Earth Women Printed Blue Kurta,33027.jpg
6,17684,Women,Footwear,Shoes,Casual Shoes,Grey,Winter,2015,Casual,Catwalk Women Ballerina Grey Casual Shoe,17684.jpg
7,3991,Men,Apparel,Topwear,Tshirts,White,Summer,2011,Casual,Mr.Men Men's Wave Rider White T-shirt,3991.jpg
8,24497,Women,Apparel,Topwear,Kurtas,Black,Fall,2011,Ethnic,Mother Earth Women Black Printed Kurta,24497.jpg
9,38457,Women,Accessories,Bags,Handbags,Navy Blue,Summer,2012,Casual,Baggit Women Navy Blue Handbag,38457.jpg


## visualize

In [6]:
df[['id','image','subCategory']].values


array([[13472, '13472.jpg', 'Dress'],
       [47539, '47539.jpg', 'Shoes'],
       [26155, '26155.jpg', 'Bottomwear'],
       ...,
       [8111, '8111.jpg', 'Belts'],
       [44379, '44379.jpg', 'Fragrance'],
       [25983, '25983.jpg', 'Fragrance']], dtype=object)

# Data preprocessing


In [7]:

from tqdm import tqdm

import numpy as np # li
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import PIL
from PIL import Image
import sys
images = []
datalabels = []

for image, subclass in df[['image','subCategory']].values:
    img = Image.open('/content/images/{}'.format(image) )
    datalabels.append(subclass)
    images.append(img)

def preprocess_image_np(image_path, desired_size=(224,224)):
    im = load_img(image_path, target_size=(224, 224))
    #im = Image.open(image_path)
    image = img_to_array(im)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    
    return image

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

N = df.shape[0]
x_train = np.empty((N, 224, 224,3), dtype=np.uint8)

for i, image_id in enumerate(tqdm(df['id'])):
    x_train[i, :, :, :] = preprocess_image_np(f'/content/images/{image_id}.jpg'
)
    

y_train = pd.get_dummies(df['subCategory']).values

print("whole splitting")
print(x_train.shape)
print(y_train.shape)

from sklearn.model_selection import train_test_split
x_train1, x_val, y_train1, y_val = train_test_split(
    x_train, y_train, 
    test_size=0.30, 
    random_state=2019
)

print("training data")
print(x_train1.shape)
print(y_train1.shape)

print("test data")
print(x_val.shape)
print(y_val.shape)

Using TensorFlow backend.


100%|██████████| 5000/5000 [00:04<00:00, 1030.72it/s]


whole splitting
(5000, 224, 224, 3)
(5000, 36)
training data
(3500, 224, 224, 3)
(3500, 36)
test data
(1500, 224, 224, 3)
(1500, 36)


# modelling

In [0]:
import tensorflow as tf

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.Session(config = config)

Num GPUs Available:  1


In [0]:
!mkdir /content/weights

In [0]:
#config = tf.ConfigProto(device_count = {'GPU': 0})
#sess = tf.Session(config=config)


In [0]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
from time import time
from keras.callbacks import TensorBoard
from math import ceil
from keras.applications.vgg19 import VGG19


# create the base pre-trained model
base_model = VGG19(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(36, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

 #first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False



model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#model4.summary()



model.fit(x_train, y_train, batch_size=32, epochs=2)


model.save('/content/weights/Vgg19_model.h5')
model.save_weights('/content/weights/Vgg19_model_weights.h5')

Epoch 1/2
5000/5000 [==============================] - 15s 3ms/step - loss: 1.1788 - acc: 0.8234
Epoch 2/2
5000/5000 [==============================] - 15s 3ms/step - loss: 0.3284 - acc: 0.9260


# Import the Pretrained models

## using MobileNetV2

In [0]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
from keras.applications.mobilenet_v2 import MobileNetV2
from time import time
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
from math import ceil

# create the base pre-trained model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(36, activation='softmax')(x)

# this is the model we will train
model1 = Model(inputs=base_model.input, outputs=predictions)

 #first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model1.summary()


# from math import ceil # import it in beginnning



model1.fit(x_train, y_train, batch_size=32, epochs=2)


model1.save('/content/weights/MobileNetV2_model.h5')
model1.save_weights('/content/weights/MobileNetV2_model_weights.h5')








9412608/9406464 [==============================] - 2s 0us/step
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
___________________________

## using Resnet50

In [0]:


from math import ceil
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
from keras.applications.resnet50 import ResNet50
from math import ceil

# create the base pre-trained model
base_model = ResNet50(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(36, activation='softmax')(x)

# this is the model we will train
model2 = Model(inputs=base_model.input, outputs=predictions)

 #first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model2.summary()


model2.fit(x_train, y_train, batch_size=32, epochs=2)

model2.save('/content/weights/Resnet50_model.h5')
model2.save_weights('/content/weights/Resnet50_model_weights.h5')

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 7s 0us/step
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________

## using VGG16

In [0]:
from keras.applications.vgg16 import VGG16
from math import ceil

# create the base pre-trained model
base_model = VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(36, activation='softmax')(x)

# this is the model we will train
model3 = Model(inputs=base_model.input, outputs=predictions)

 #first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model3.summary()


model3.fit(x_train, y_train, batch_size=32, epochs=2)

model3.save('/content/weights/Vgg16_model.h5')
model3.save_weights('/content/weights/Vgg16_model_weights.h5')


58892288/58889256 [==============================] - 5s 0us/step
Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

## using VGG19

In [0]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
from time import time
from keras.callbacks import TensorBoard
from math import ceil
from keras.applications.vgg19 import VGG19


# create the base pre-trained model
base_model = VGG19(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(36, activation='softmax')(x)

# this is the model we will train
model4 = Model(inputs=base_model.input, outputs=predictions)

 #first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

model4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model4.summary()


model4.fit(x_train, y_train, batch_size=32, epochs=2)
model4.save('/content/weights/Vgg19_2_model.h5')
model4.save_weights('/content/weights/Vgg19_2_model_weights.h5')



Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

## using Xception

In [0]:
from keras.applications.xception import Xception
from time import time
from keras.callbacks import TensorBoard
from math import ceil

# create the base pre-trained model
base_model = VGG19(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(36, activation='softmax')(x)

# this is the model we will train
model5 = Model(inputs=base_model.input, outputs=predictions)

 #first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

model5.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model5.summary()

import keras
from matplotlib import pyplot as plt
#history = model5.fit(train_x, train_y,validation_split = 0.1, epochs=50, batch_size=4)

model5.fit(x_train, y_train, batch_size=32, epochs=2)


model5.save('/content/weights/Xception_model.h5')
model5.save_weights('/content/weights/Xception_model_weights.h5')


Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

## Inception V3

In [0]:

from math import ceil
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
from keras.applications.inception_v3 import InceptionV3


from math import ceil

# create the base pre-trained model
base_model = InceptionV3(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(36, activation='softmax')(x)

# this is the model we will train
model6 = Model(inputs=base_model.input, outputs=predictions)

 #first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

model6.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model6.summary()


model6.fit(x_train, y_train, batch_size=32, epochs=2)

model6.save('/content/weights/InceptionV3_model.h5')
model6.save_weights('/content/weights/InceptionV3_model_weights.h5')


87916544/87910968 [==============================] - 7s 0us/step
Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 111, 111, 32) 864         input_8[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 111, 111, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_50 (Activation)      (None, 111, 111, 32) 0           batch_normalization_1[0][0]      
__________________________

# Choose the Best Model Among the Above based on Accuracy!

Accuracies of The 5 Best Models are as follows:(1st training)
* **MobileNetV2 :**  acc: 94%         val: 9%
* **ResNet50 :**     acc:   94%       val:  67%
* **VGG16:**      acc:     79%     val:  80%
* **VGG19:**       acc:      89%    val: 88%
* **Xception:**       acc:     78%     val: 79%
* **inception V3:**       acc:     85%     val: 82%

NewAccuracies of The 5 Best Models are as follows:((2nd training)
* **MobileNetV2 :**  acc: 94%         val: 9%
* **ResNet50 :**     acc:   94%       val:  62%
* **VGG16:**      acc:     86%     val:  85%
* **VGG19:**       acc:      91%    val: 88%
* **Xception:**       acc:     72%     val: 86%
* **inception V3:**       acc:     85%      val: 80% 

NewAccuracies of The 5 Best Models are as follows:((3rd training)
* **MobileNetV2 :**  acc: 94%         val: 9%
* **ResNet50 :**     acc:   94%       val:  62%
* **VGG16:**      acc:     86%     val:  85%
* **VGG19:**       acc:      91%    val: 88%
* **Xception:**       acc:     72%     val: 86%
* **inception V3:**       acc:     85%      val: 81%

## **VGG19 performs best** with **Acc:91%** and **VACC:88%**

# save all models and weights in zip file for future use

In [0]:
!zip -r /content/myresultingzips.zip /content/weights


  adding: content/weights/ (stored 0%)
  adding: content/weights/Resnet50_model.h5 (deflated 7%)
  adding: content/weights/Vgg19_2_model.h5 (deflated 7%)
  adding: content/weights/Xception_model.h5 (deflated 7%)
  adding: content/weights/Xception_model_weights.h5 (deflated 7%)
  adding: content/weights/Vgg19_model_weights.h5 (deflated 7%)
  adding: content/weights/Resnet50_model_weights.h5 (deflated 8%)
  adding: content/weights/MobileNetV2_model_weights.h5 (deflated 9%)
  adding: content/weights/InceptionV3_model_weights.h5 (deflated 8%)
  adding: content/weights/Vgg19_2_model_weights.h5 (deflated 7%)
  adding: content/weights/Vgg16_model_weights.h5 (deflated 7%)
  adding: content/weights/Vgg16_model.h5 (deflated 7%)
  adding: content/weights/Vgg19_model.h5 (deflated 7%)
  adding: content/weights/InceptionV3_model.h5 (deflated 8%)
  adding: content/weights/MobileNetV2_model.h5 (deflated 8%)


# Using the weights of the **Best Model out of 6 models** , lets predict an image and give (label,probability)

In [11]:
# example of using a pre-trained model as a classifier
from keras.preprocessing.image import load_img
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
import json 
# load an image from file
image = load_img('/content/sample.jpg', target_size=(224, 224))
# convert the image pixels to a numpy array
image = img_to_array(image)
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# prepare the image for the VGG model
image = preprocess_input(image)
# load the model
model = load_model('/content/weights/Vgg19_model.h5')
#model = VGG16()
# predict the probability across all output classes
preds = model.predict(image)


top = 5


labelresults = []
for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [datalabels[i] + str(pred[i],) for i in top_indices]
        #result.sort(key=lambda x: x[2], reverse=True)
        labelresults.append(result)
        
# convert the probabilities to class labels
label = labelresults
# retrieve the most likely result, e.g. highest probability
label = label[0][0]
print(label)













Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Bags0.86231625


# predicting labels for test images

In [39]:
preds = model.predict(x_val)

top = 5  # 5 highest probable labels

alllabelresults = []
allpredictedlabels = []
for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [datalabels[i] + str(pred[i],) for i in top_indices]
        #result.sort(key=lambda x: x[2], reverse=True)
        alllabelresults.append(result)
        

# convert the probabilities to class labels and  retrieve the most likely result, e.g. highest probability
for label in alllabelresults:
    allpredictedlabels.append(label[0])
    print(label[0])


Watches0.9943059
Bottomwear0.9997248
Bags0.99986434
Watches0.9998704
Bottomwear0.9924642
Bags0.9986872
Shoes0.9973127
Bags0.9999995
Bags0.9992861
Topwear0.9935401
Bottomwear0.99876827
Bags0.88444203
Bags0.99889237
Bags0.99733424
Bags0.9722578
Watches0.99999666
Bags0.99999976
Bags0.9999943
Bottomwear0.998262
Watches0.9988545
Bags0.99994874
Bags0.84658843
Topwear0.96410894
Bags0.98550713
Bags0.99871254
Bags0.9999939
Bottomwear0.9827907
Shoes0.8654757
Watches0.99994385
Shoes0.396372
Watches0.99979
Watches1.0
Wallets0.99994826
Bottomwear0.9999939
Wallets0.99617696
Shoes0.99876875
Bottomwear0.99943215
Bags0.9937988
Shoes0.8736601
Topwear0.6036459
Bags0.99996984
Bags0.99977857
Bags0.8554654
Shoes0.9500321
Bottomwear0.99995244
Bags0.9994357
Bags0.9999856
Bags0.98282987
Topwear0.9987834
Bags0.9829675
Bags0.9971215
Bottomwear0.99990356
Bottomwear0.99978393
Bottomwear0.6527241
Bags0.9993962
Bags0.99985576
Topwear0.9999943
Bags0.9998988
Bottomwear0.99950826
Bottomwear0.6848196
Bottomwear0.9995523

In [41]:
submission = pd.DataFrame({ 'imageId': df[3500:].image.values, 'label': allpredictedlabels })
submission.to_csv("my_submission.csv", index=False)
submission

,imageId,label
0,30857.jpg,Watches0.9943059
1,19584.jpg,Bottomwear0.9997248
2,33825.jpg,Bags0.99986434
3,40545.jpg,Watches0.9998704
4,29522.jpg,Bottomwear0.9924642
...,...,...
1495,48343.jpg,Topwear0.9999151
1496,48917.jpg,Bottomwear0.99000543
1497,8111.jpg,Bags0.9999999
1498,44379.jpg,Bags0.9999604


# conclusion / The results:

* I have used 6 models to see which performs best on the given dataset
* The Vgg19 Performed very well on the given dataset with accuracy(91%) and validation_acc(88%)
* Therefore , I have used Its weights(Vgg19_model_weights) to predict the label of images

#  Thanks  and suggestions for  improvements are always  welcome
#  satyamsharma.